In [1]:
import tensorflow as tf

In [4]:
from tensorflow.keras.layers import Input,Dense,Flatten
from tensorflow.keras.applications.vgg16 import VGG16 as PretrainedModel,preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import SGD,Adam
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from glob import glob
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys, os

In [10]:
# Data from: https://mmspg.epfl.ch/dowloads/food-image-datasets/
!wget --passive-ftp --prefer-family=ipv4 --ftp-user FoodImage@grebvm2.epfl.ch \
 --ftp-password Cahclmoo -nc ftp://tremplin.epfl.ch/Food-5K.zip

--2025-02-28 12:10:53--  ftp://tremplin.epfl.ch/Food-5K.zip
           => ‘Food-5K.zip’
Resolving tremplin.epfl.ch (tremplin.epfl.ch)... 128.178.218.41, 2001:620:618:1da:7:80b2:da01:3
Connecting to tremplin.epfl.ch (tremplin.epfl.ch)|128.178.218.41|:21... connected.
Logging in as FoodImage@grebvm2.epfl.ch ... 
Error in server response, closing control connection.
Retrying.

--2025-02-28 12:13:02--  ftp://tremplin.epfl.ch/Food-5K.zip
  (try: 2) => ‘Food-5K.zip’
Connecting to tremplin.epfl.ch (tremplin.epfl.ch)|128.178.218.41|:21... connected.
Logging in as FoodImage@grebvm2.epfl.ch ... 
Error in server response, closing control connection.
Retrying.

--2025-02-28 12:15:12--  ftp://tremplin.epfl.ch/Food-5K.zip
  (try: 3) => ‘Food-5K.zip’
Connecting to tremplin.epfl.ch (tremplin.epfl.ch)|128.178.218.41|:21... connected.
Logging in as FoodImage@grebvm2.epfl.ch ... 
Error in server response, closing control connection.
Retrying.

--2025-02-28 12:17:23--  ftp://tremplin.epfl.ch/Food-5K.zip
 

In [11]:
! unzip -qq -o Food-5K.zip

unzip:  cannot find or open Food-5K.zip, Food-5K.zip.zip or Food-5K.zip.ZIP.


In [12]:
!ls

sample_data


In [13]:
!ls training

ls: cannot access 'training': No such file or directory


In [14]:
# look at an image for fun
plt.imshow(image.load_img('training/0_100.jpg'))
plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'training/0_100.jpg'

In [ ]:
# Food images start with 1, non-food images start with 0
plt.imshow(image.load_img('training/1_100.jpg'))
plt.show()

In [ ]:
!mkdir data

In [ ]:
# Make directories to store the data Keras-style
!mkdir data/train
!mkdir data/test
!mkdir data/train/nonfood
!mkdir data/train/food
!mkdir data/test/nonfood
!mkdir data/test/food

In [ ]:
# Move the images
# Note we will consider 'training' to be the train set
#      'validation' folder will be the test set
#      ignore the 'evaluation' set
!mv training/0*.jpg data/train/nonfood
!mv training/1*.jpg data/train/food
!mv validation/0*.jpg data/test/nonfood
!mv validation/1*.jpg data/test/food

In [ ]:
train_path = 'data/train'
valid_path = 'data/test'

In [ ]:
# These images are pretty big and of different sizes
# Let's load them all in as the same (smaller) size
IMAGE_SIZE =[200,200]

In [ ]:
# useful for getting number of files
image_files = glob(train_path + '/*/*.jpg')
valid_image_files = glob(valid_path + '/*/*.jpg')

In [ ]:
# useful for getting number of classes
folders = glob(train_path + '/*')
folders

In [ ]:
# look at an image for fun
plt.imshow(image.load_img(np.random.choice(image_files)))
plt.show()

In [ ]:
ptm = PretrainedModel(
    input_shape=IMAGE_SIZE + [3],
    weights='imagenet',
    include_top=False)

In [ ]:
# freeze pretrainde model wights
ptm.trainable = False

In [ ]:
# map the data into feature vectors

# Keras image data generator returns classes one-hot encoded

K = len(folders) # number of classes
x = Flatten()(ptm.output)
x = Dense(K, activation='softmax')(x)

In [ ]:
#create a model object
model = Model(inputs=ptm.input,outputs=x)

In [ ]:
#view the structure of the model
model.summary()

In [ ]:
#create an instance of ImageDataGenerator
gen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    preprocessing_function=preprocess_input)


In [ ]:
batch_size = 128

# create generators
train_generator = gen.flow_from_directory(
    train_path,
    shuffle=True,
    target_size=IMAGE_SIZE,
    batch_size=batch_size,
)
valid_generator = gen.flow_from_directory(
    valid_path,
    target_size=IMAGE_SIZE,
    batch_size=batch_size,
)

In [ ]:
model.compile(
    loss='categorical_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

In [ ]:
# fit the model
r = model.fit_generator(
    train_generator,
    validation_data=valid_generator,
    epochs=10
    steps_per_epoch = int(np.ceil(len(image_files)/batch_size)),
    validation_steps = int(np.ceil(len(valid_image_files)/batch_size))
)

In [ ]:
# loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.show()

In [ ]:
# accuracies
plt.plot(r.history['accuracy'], label='train acc')
plt.plot(r.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()